In [4]:
from numpy import *

#读取文本文件
def loadDataSet(fileName):
    dataMat = []
    labelMat = []
    #print(type(dataMat))
    fr = open(fileName)
    for line in fr.readlines():
        lineArr = line.strip().split()
        dataMat.append([1.0, float(lineArr[0]), float(lineArr[1])])
        labelMat.append(int(lineArr[2]))
    return dataMat, labelMat

#loadDataSet('testSet.txt')
#########################################################################################
#sigmoid函数
def sigmoid(inX):
    return 1.0/(1.0 + exp(-inX))

#sigmoid(0)
#########################################################################################
#梯度上升算法, alpha:每次移动的步长0.001， maxCycles:迭代次数500
def gradAscent(dataMatIn, classLabels, alpha, maxCycles):
    dataMatrix = mat(dataMatIn)
    labelMat = mat(classLabels).transpose()
    m, n = shape(dataMatrix)
    weights = ones((n, 1))  #生成列向量，用于后面的矩阵乘法
    for k in range(maxCycles):
        h = sigmoid(dataMatrix * weights)
        error = (labelMat - h)
        weights = weights + alpha * dataMatrix.transpose() * error
    return weights
    
########################################################################################
#画出数据集和logistics回归最佳拟合直线的函数
def plotBestFit(weights):
    import matplotlib.pyplot as plt
    dataMat, labelMat = loadDataSet('testSet.txt')
    dataArr = array(dataMat)
    n = shape(dataArr)[0]
    xcord1 = []
    ycord1 = []
    xcord2 = []
    ycord2 = []
    for i in range(n):
        if int(labelMat[i]) == 1:
            xcord1.append(dataArr[i, 1])
            ycord1.append(dataArr[i, 2])
        else:
            xcord2.append(dataArr[i, 1])
            ycord2.append(dataArr[i, 2])
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.scatter(xcord1, ycord1, s = 30, c = 'red', marker = '*')
    ax.scatter(xcord2, ycord2, s = 30, c = 'green')
    x = arange(-3, 3, 0.1)
    y = transpose((-weights[0]-weights[1]*x)/weights[2])
    ax.plot(x,y)
    plt.xlabel('x1')
    plt.ylabel('x2')
    plt.show()

##############################################################################################
#随机梯度上升算法 alpha = 0.01
def stocGradAscent0(dataMatrix, classLabels, alpha):
    m, n = shape(dataMatrix)
    weights = ones(n)                    #后面不是矩阵乘法，所以不用生成列向量
    for i in range(m):
        h = sigmoid(sum(dataMatrix[i] * weights))
        error = classLabels[i] - h
        weights = weights + alpha * error * dataMatrix[i]
    return weights

###############################################################################################
#改进后的随机梯度上升算法 numIter :循环次数  默认迭代150次，除非给定参数
def stocGradAscent1(dataMatrix, classLabels, numIter=150):
    m, n = shape(dataMatrix)
    weights = ones(n)                    #后面不是矩阵乘法，所以不用生成列向量
    for j in range(numIter):
        dataIndex = list(range(m))              #dataIndex = 0,1,2,.....99,range产生的是一个迭代器，其元素不可以删除，现将其转为一个列表
        for i in range(m):
            alpha = 4/(1.0+j+i)+0.01       #alpha会随着迭代次数不断减小
            randIndex = int(random.uniform(0, len(dataIndex))) #随机选择样本   
            h = sigmoid(sum(dataMatrix[randIndex] * weights))
            error = classLabels[randIndex] - h
            weights = weights + alpha * error * dataMatrix[randIndex]
            del(dataIndex[randIndex])
    return weights

################################################################################################
#logistic回归分类
def classify(inX, weights):
    prob = sigmoid(sum(inX * weights))
    if prob > 0.5:
        return 1.0
    else:
        return 0.0
    
def colicTest(trainFileName, testFileName):
    frTrain = open(trainFileName)
    frTest = open(testFileName)
    trainSet = []
    trainLabels = []
    for line in frTrain.readlines():
        currLine = line.strip().split('\t')
        lineArr = []
        for i in range(21):  #21维特征
            lineArr.append(float(currLine[i]))
        trainSet.append(lineArr)
        trainLabels.append(float(currLine[21]))
    trainWeights = stocGradAscent1(array(trainSet), trainLabels, 10)
    errorCount = 0
    numTestVec = 0
    for line in frTest.readlines():
        numTestVec += 1.0
        currLine = line.strip().split('\t')
        lineArr = []
        for i in range(21):
            lineArr.append(float(currLine[i]))
        if int(classify(array(lineArr), trainWeights)) != int(lineArr[21]):
            errorCount += 1.0
    errorRate = float(errorCount)/numTestVec
    print('the error rate of this test is: %f' % errorRate)
    return errorRate

def multiTest(trainFileName, testFileName):
    numTests = 10
    errorSum = 0
    for k in range(numTests):
        errorSum += colicTest(trainFileName, testFileName)
    print('after %d iterations the average error rate is: %f' % (numTests, errorSum/float(numTests)))
        
    